<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Unit%203%20Reservoir%20Statics/notebook/3_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 3 Reservoir Statics (Examples)**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 193 (delta 13), reused 0 (delta 0), pack-reused 158
Receiving objects: 100% (193/193), 221.18 KiB | 636.00 KiB/s, done.
Resolving deltas: 100% (80/80), done.


In [0]:
import sys, os
sys.path.append('/content/reservoir-engineering/Unit 3 Reservoir Statics/functions')

# Example 3.1 Calculating Oil Density in a Reservoir from Well Test Data

In [28]:
"Question. calculate oil density in the reservoir based on well data"

# known

qo = 250 # in STB/day
qg = 150 # Mscf/day
api = 45
sg = 0.82
temp = 240 # in fahrenheit

# standard conditions

temp_sc = 60 # in fahrenheit
pressure_sc = 14.73 # 1 atm = 14.73 psia
z_sc = 1 # gas z factor at standard condition

# calculate gas-oil ratio

Rs = qg * 1000 / qo # in scf gas/STB oil
Rs_converted = Rs * (1 / 5.6148) # convert to scf gas/scf oil
print("Gas-oil ratio:", Rs_converted, "scf gas/scf oil")

# oil FVF at surface/standard condition using Standing correlation (Equation 2.37 See Unit 2)

so = 141.5 / (api + 131.5)
F = Rs * ((sg / so)**0.5) + (1.25 * temp_sc) # Rs must be in scf/STB
Bo = 0.972 + (0.000147*(F**1.1756))
print("Oil FVF at standard condition:", Bo, "RB/STB")

# oil density at surface/standard condition

rhowater = 62.366 # 1 g/cm3 = 62.366 lbm/ft3
rhooil_sc = so * rhowater
print("Oil density at standard condition:", rhooil_sc, "lbm/ft3")

# gas density at surface/standard condition (Eq 2.23, real-gas law)

R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R)
rhogas_sc = (28.97 * sg * pressure_sc) / (z_sc * R * (temp_sc + 459)) # temp converted to Rankine
print("Gas density at standard condition:", rhogas_sc, "lbm/ft3")

# finally, oil density at reservoir condition

rhooil = (rhooil_sc + (rhogas_sc * Rs_converted)) / Bo
print("Oil density at reservoir condition:", rhooil, "lbm/ft3")

# oil density gradient

rhooil_grad = rhooil / 144 # 144 is factor conversion from density lbm/ft3 to psi/ft
print("Oil density gradient:", rhooil_grad, "psi/ft")

Gas-oil ratio: 106.8604402650139 scf gas/scf oil
Oil FVF at standard condition: 1.2871893623377872 RB/STB
Oil density at standard condition: 49.99880453257791 lbm/ft3
Gas density at standard condition: 0.06282276870641311 lbm/ft3
Oil density at reservoir condition: 44.05884240078876 lbm/ft3
Oil density gradient: 0.30596418333881087 psi/ft


# Example 3.2 Investigating Whether Two Oil Wells Are in Pressure Communication

In [43]:
# Question. Are the two wells in pressure communication?

# known

api = 32
Rs = 500 # in scf/STB
temp = 200 # in fahrenheit
sg = 0.72
Bo = 1.3 # in RB/STB

well1_elev = 7134; well2_elev = 7028 # well elevation in ft
well1_depthkb = 5652; well2_depthkb = 5426 # well depth from KB in ft
well1_pressure = 2453; well2_pressure = 2306 # well pressure in psia

# standard conditions

temp_sc = 60 # in fahrenheit
pressure_sc = 14.73 # 1 atm = 14.73 psia
z_sc = 1 # gas z factor at standard condition

# calculate oil density at standard condition

so = 141.5 / (131.5 + api)
rhowater = 62.366
rhooil_sc = so * rhowater
# print("Oil density at standard condition:", rhooil_sc, "lbm/ft3")

# calculate gas density at standard condition

R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R)
rhogas_sc = (28.97 * sg * pressure_sc) / (z_sc * R * (temp_sc + 459)) # temp converted to Rankine
# print("Gas density at standard condition:", rhogas_sc, "lbm/ft3")

# calculate oil density at reservoir condition

Rs_converted = Rs * (1 / 5.6148) # convert to scf gas/scf oil
rhooil = (rhooil_sc + (rhogas_sc * Rs_converted)) / Bo
print("Oil density at reservoir condition:", rhooil, "lbm/ft3")

# oil density gradient

rhooil_grad = rhooil / 144 # 144 is factor conversion from density lbm/ft3 to psi/ft
# print("Oil density gradient:", rhooil_grad, "psi/ft")

# check if the two wells are communicated

well1_depthmsl = well1_elev - well1_depthkb
well2_depthmsl = well2_elev - well2_depthkb
delta_depth = np.abs(well2_depthmsl - well1_depthmsl)

if well1_depthmsl > well2_depthmsl:
  print("well 1 is shallower", delta_depth, "ft than well 2")
else:
  print("well 1 is deeper", delta_depth, "ft than well 2")

well1_pressure_extrapolated = well2_pressure + rhooil_grad * delta_depth # pressure extrapolation, Eq 3.1
print("From extrapolation, well 1 is supposed to be", well1_pressure_extrapolated, "psi")

delta_extrapolated_actual = np.abs(well1_pressure_extrapolated - well1_pressure)
percent_delta = delta_extrapolated_actual / well1_pressure * 100

print("Difference of extrapolated well 1 pressure to actual pressure:", delta_extrapolated_actual, "psi, or percent difference:", percent_delta, "%")

if percent_delta < 1:
  print("The two wells are most likely in pressure communication")
else:
  print("The two wells are most likely NOT in pressure communication")

Oil density at reservoir condition: 45.29722499549856 lbm/ft3
well 1 is deeper 120 ft than well 2
From extrapolation, well 1 is supposed to be 2343.747687496249 psi
Difference of extrapolated well 1 pressure to actual pressure: 109.252312503751 psi, or percent difference: 4.45382439884839 %
The two wells are most likely NOT in pressure communication
